In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


# import data

In [2]:
data = pd.read_pickle("outputs_new/dedupe_data_group_merge.pkl")
data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084855,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None,parent,None,None
8084856,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None,parent,None,None
8084857,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None,parent,None,None
8084858,openalex::W999790414,viXra,open

# build parent_index and long_full file

In [3]:
# import os
# import json
# import duckdb
# import pandas as pd

# def build_parent_index_and_long_full_duckdb(
#     df: pd.DataFrame,
#     *,
#     out_dir: str = "outputs",
#     record_id_col: str = "record_id",
#     server_col: str = "server_name",
#     doi_col: str = "doi",
#     url_col: str = "landing_page_url",
#     title_col: str = "title",
#     authors_col: str = "authors_flat",
#     date_col: str = "date_first_seen",
#     year_col: str = "publication_year_first_seen",
#     group_id_col: str = "dup_group_id",
#     # outputs
#     long_full_name: str = "dedupe_clusters_long_full.parquet",
#     parent_index_name: str = "parent_version_index_full.parquet",
#     metrics_name: str = "dedupe_full_metrics.json",
# ):
#     """
#     Rules:
#       - singleton rows: parent = itself
#       - clustered rows: parent = oldest in cluster (date_col ASC, tie record_id ASC)
#       - group id for full universe: dup_group_id_full = COALESCE(dup_group_id, record_id)

#     Outputs (in out_dir):
#       - dedupe_clusters_long_full.parquet  (ALL rows, includes parent_record_id_selected, is_parent_selected, group_size)
#       - parent_version_index_full.parquet  (ONE row per group, with parent info + members lists)
#       - dedupe_full_metrics.json
#     """
#     os.makedirs(out_dir, exist_ok=True)
#     con = duckdb.connect(database=":memory:")
#     con.execute("PRAGMA threads=8;")          # tune if needed
#     con.execute("PRAGMA memory_limit='16GB';")# tune if needed

#     # Register dataframe
#     con.register("df_in", df)

#     # -------------
#     # 0) Build a stable "full" group id (clusters + singletons)
#     # -------------
#     # Note: we compute parent per group using window functions.
#     # Parent selection: MIN_BY(record_id, sort_key) doesn't exist in DuckDB,
#     # but we can do FIRST_VALUE over an ordered window.
#     con.execute(f"""
#         CREATE OR REPLACE TEMP VIEW base AS
#         SELECT
#             *,
#             COALESCE(CAST({group_id_col} AS VARCHAR), CAST({record_id_col} AS VARCHAR)) AS dup_group_id_full,
#             TRY_CAST({date_col} AS TIMESTAMP) AS dt,
#             TRY_CAST({record_id_col} AS BIGINT) AS rid_num,
#             NULLIF(TRIM(CAST({doi_col} AS VARCHAR)), '') AS doi_clean,
#             NULLIF(TRIM(CAST({server_col} AS VARCHAR)), '') AS server_clean
#         FROM df_in
#     """)

#     # -------------
#     # 1) Create long_full (ALL rows)
#     #    parent_record_id_selected = oldest record_id within dup_group_id_full
#     # -------------
#     con.execute(f"""
#         CREATE OR REPLACE TEMP VIEW long_full AS
#         SELECT
#             -- keep original columns
#             b.*,

#             -- selected parent per group
#             FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_record_id_selected,

#             -- parent metadata (from the chosen parent row)
#             FIRST_VALUE(CAST({server_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_server_name_selected,

#             FIRST_VALUE(CAST({doi_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_doi_selected,


#             -- add ------------
#             FIRST_VALUE(CAST({url_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_url_selected,
            
#             FIRST_VALUE(CAST({title_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_title_selected,
            
#             FIRST_VALUE(CAST({authors_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_authors_selected,

#             -- ---------
    

#             FIRST_VALUE(dt) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_date_selected,

#             FIRST_VALUE(CAST({year_col} AS VARCHAR)) OVER (
#                 PARTITION BY dup_group_id_full
#                 ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#             ) AS parent_year_selected,

#             -- is this row the selected parent?
#             CASE
#               WHEN CAST({record_id_col} AS VARCHAR) =
#                    FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
#                       PARTITION BY dup_group_id_full
#                       ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
#                    )
#               THEN TRUE ELSE FALSE
#             END AS is_parent_selected,

#             COUNT(*) OVER (PARTITION BY dup_group_id_full) AS group_size
#         FROM base b
#     """)

#     # Export full long table
#     long_path = os.path.join(out_dir, long_full_name)
#     con.execute(f"COPY long_full TO '{long_path}' (FORMAT 'parquet');")

#     # -------------
#     # 2) Build parent_version_index_full (ONE row per group)
#     #    Includes lists (record_ids, dois, servers with counts), first/last version dates, etc.
#     # -------------
#     # a) servers with counts as "ServerA (12); ServerB (3)"
#     con.execute(f"""
#         CREATE OR REPLACE TEMP VIEW servers_counts AS
#         SELECT
#             dup_group_id_full,
#             CAST({server_col} AS VARCHAR) AS server_name,
#             COUNT(*) AS n
#         FROM long_full
#         GROUP BY 1, 2
#     """)

#     con.execute("""
#         CREATE OR REPLACE TEMP VIEW servers_counts_str AS
#         SELECT
#             dup_group_id_full,
#             STRING_AGG(server_name || ' (' || CAST(n AS VARCHAR) || ')', '; ' ORDER BY n DESC, server_name ASC) AS servers_with_counts
#         FROM servers_counts
#         GROUP BY 1
#     """)

#     # b) most recent version server(s): if multiple on same max date → join with ';'
#     con.execute("""
#         CREATE OR REPLACE TEMP VIEW max_dt AS
#         SELECT dup_group_id_full, MAX(dt) AS max_dt
#         FROM long_full
#         GROUP BY 1
#     """)

#     con.execute("""
#         CREATE OR REPLACE TEMP VIEW most_recent_servers AS
#         SELECT
#             l.dup_group_id_full,
#             STRING_AGG(DISTINCT CAST(l.{server_col} AS VARCHAR), '; ' ORDER BY CAST(l.{server_col} AS VARCHAR)) AS most_recent_server_names
#         FROM long_full l
#         JOIN max_dt m
#           ON l.dup_group_id_full = m.dup_group_id_full
#          AND l.dt = m.max_dt
#         GROUP BY 1
#     """)

#     # c) parent index
#     parent_path = os.path.join(out_dir, parent_index_name)
#     con.execute(f"""
#         COPY (
#             SELECT
#                 dup_group_id_full,

#                 -- parent (selected oldest)
#                 parent_record_id_selected AS parent_record_id,
#                 parent_server_name_selected AS parent_server_name,
#                 parent_doi_selected       AS parent_doi,
#                 parent_url_selected       AS parent_url,
#                 parent_title_selected       AS parent_title,
#                 parent_authors_selected       AS parent_authors,
#                 parent_date_selected      AS parent_date_first_seen,
#                 parent_year_selected      AS parent_year_first_seen,

#                 -- cluster members lists (ALL of them, not just 30)
#                 STRING_AGG(DISTINCT CAST({record_id_col} AS VARCHAR), '; ' ORDER BY CAST({record_id_col} AS VARCHAR)) AS version_record_ids,
#                 STRING_AGG(DISTINCT CAST({doi_col} AS VARCHAR), '; ' ORDER BY CAST({doi_col} AS VARCHAR)) FILTER (WHERE {doi_col} IS NOT NULL AND CAST({doi_col} AS VARCHAR) <> '') AS version_dois,

#                 -- server list with counts
#                 s.servers_with_counts,

#                 -- size + first/last version date
#                 MAX(group_size) AS total_versions,
#                 MIN(dt) AS first_version_date,
#                 MAX(dt) AS last_version_date,

#                 -- most recent version server(s)
#                 mr.most_recent_server_names AS most_recent_version_servers

#             FROM long_full l
#             LEFT JOIN servers_counts_str s USING(dup_group_id_full)
#             LEFT JOIN most_recent_servers mr USING(dup_group_id_full)
#             GROUP BY
#                 dup_group_id_full,
#                 parent_record_id_selected,
#                 parent_server_name_selected,
#                 parent_doi_selected,
#                 parent_url_selected,
#                 parent_title_selected,
#                 parent_authors_selected,
#                 parent_date_selected,
#                 parent_year_selected,
#                 s.servers_with_counts,
#                 mr.most_recent_server_names
#         )
#         TO '{parent_path}' (FORMAT 'parquet');
#     """)

#     # -------------
#     # 3) Metrics
#     # -------------
#     total_rows = con.execute("SELECT COUNT(*) FROM base").fetchone()[0]
#     clustered_rows = con.execute(f"SELECT COUNT(*) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
#     singleton_rows = total_rows - clustered_rows

#     clusters = con.execute(f"SELECT COUNT(DISTINCT CAST({group_id_col} AS VARCHAR)) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
#     groups_full = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM base").fetchone()[0]

#     # sanity checks
#     long_rows = con.execute("SELECT COUNT(*) FROM long_full").fetchone()[0]
#     parent_rows = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM long_full").fetchone()[0]

#     metrics = {
#         "total_rows": int(total_rows),
#         "clustered_rows_has_dup_group_id": int(clustered_rows),
#         "singleton_rows_no_dup_group_id": int(singleton_rows),
#         "clusters_distinct_dup_group_id": int(clusters),
#         "groups_full_clusters_plus_singletons": int(groups_full),
#         "long_full_rows_written": int(long_rows),
#         "parent_index_rows_written": int(parent_rows),
#     }

#     # write metrics json
#     metrics_path = os.path.join(out_dir, metrics_name)
#     with open(metrics_path, "w", encoding="utf-8") as f:
#         json.dump(metrics, f, indent=2)

#     return {
#         "long_full_path": long_path,
#         "parent_index_path": parent_path,
#         "metrics_path": metrics_path,
#         "metrics": metrics,
#     }


In [4]:
import os
import json
import duckdb
import pandas as pd

def build_parent_index_and_long_full_duckdb(
    df: pd.DataFrame,
    *,
    out_dir: str = "outputs",
    record_id_col: str = "record_id",
    server_col: str = "server_name",
    doi_col: str = "doi",
    url_col: str = "landing_page_url",
    title_col: str = "title",
    authors_col: str = "authors_flat",
    date_col: str = "date_first_seen",
    year_col: str = "publication_year_first_seen",
    group_id_col: str = "dup_group_id",
    # outputs
    long_full_name: str = "dedupe_clusters_long_full.parquet",
    parent_index_name: str = "parent_version_index_full.parquet",
    metrics_name: str = "dedupe_full_metrics.json",
):
    """
    Rules:
      - singleton rows: parent = itself
      - clustered rows: parent = oldest in cluster (date_col ASC, tie record_id ASC)
      - group id for full universe: dup_group_id_full = COALESCE(dup_group_id, record_id)

    Outputs (in out_dir):
      - dedupe_clusters_long_full.parquet  (ALL rows, includes parent_record_id_selected, is_parent_selected, group_size)
      - parent_version_index_full.parquet  (ONE row per group, with parent info + members lists)
      - dedupe_full_metrics.json
    """
    os.makedirs(out_dir, exist_ok=True)
    con = duckdb.connect(database=":memory:")
    con.execute("PRAGMA threads=8;")          # tune if needed
    con.execute("PRAGMA memory_limit='16GB';")# tune if needed

    # Register dataframe
    con.register("df_in", df)

    # -------------
    # 0) Build a stable "full" group id (clusters + singletons)
    # -------------
    # Note: we compute parent per group using window functions.
    # Parent selection: MIN_BY(record_id, sort_key) doesn't exist in DuckDB,
    # but we can do FIRST_VALUE over an ordered window.
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW base AS
        SELECT
            *,
            COALESCE(CAST({group_id_col} AS VARCHAR), CAST({record_id_col} AS VARCHAR)) AS dup_group_id_full,
            TRY_CAST({date_col} AS TIMESTAMP) AS dt,
            TRY_CAST({record_id_col} AS BIGINT) AS rid_num,
            NULLIF(TRIM(CAST({doi_col} AS VARCHAR)), '') AS doi_clean,
            NULLIF(TRIM(CAST({server_col} AS VARCHAR)), '') AS server_clean
        FROM df_in
    """)

    # -------------
    # 1) Create long_full (ALL rows)
    #    parent_record_id_selected = oldest record_id within dup_group_id_full
    # -------------
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW long_full AS
        SELECT
            -- keep original columns
            b.*,

            -- selected parent per group
            FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_record_id_selected,

            -- parent metadata (from the chosen parent row)
            FIRST_VALUE(CAST({server_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_server_name_selected,

            FIRST_VALUE(CAST({doi_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_doi_selected,


            -- add ------------
            FIRST_VALUE(CAST({url_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_url_selected,
            
            FIRST_VALUE(CAST({title_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_title_selected,
            
            FIRST_VALUE(CAST({authors_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_authors_selected,

            -- ---------
    

            FIRST_VALUE(dt) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_date_selected,

            FIRST_VALUE(CAST({year_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_year_selected,

            -- is this row the selected parent?
            CASE
              WHEN CAST({record_id_col} AS VARCHAR) =
                   FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
                      PARTITION BY dup_group_id_full
                      ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
                   )
              THEN TRUE ELSE FALSE
            END AS is_parent_selected,

            COUNT(*) OVER (PARTITION BY dup_group_id_full) AS group_size
        FROM base b
    """)

    # Export full long table
    long_path = os.path.join(out_dir, long_full_name)
    con.execute(f"COPY long_full TO '{long_path}' (FORMAT 'parquet');")

    # -------------
    # 2) Build parent_version_index_full (ONE row per group)
    #    Includes lists (record_ids, dois, servers with counts), first/last version dates, etc.
    # -------------
    # a) servers with counts as "ServerA (12); ServerB (3)"
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW servers_counts AS
        SELECT
            dup_group_id_full,
            CAST({server_col} AS VARCHAR) AS server_name,
            COUNT(*) AS n
        FROM long_full
        GROUP BY 1, 2
    """)

    con.execute("""
        CREATE OR REPLACE TEMP VIEW servers_counts_str AS
        SELECT
            dup_group_id_full,
            STRING_AGG(server_name || ' (' || CAST(n AS VARCHAR) || ')', '; ' ORDER BY n DESC, server_name ASC) AS servers_with_counts
        FROM servers_counts
        GROUP BY 1
    """)

    # b) most recent version server(s): if multiple on same max date → join with ';'
    con.execute("""
        CREATE OR REPLACE TEMP VIEW max_dt AS
        SELECT dup_group_id_full, MAX(dt) AS max_dt
        FROM long_full
        GROUP BY 1
    """)

    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW most_recent_servers AS
        SELECT
            l.dup_group_id_full,
            STRING_AGG(DISTINCT CAST(l.{server_col} AS VARCHAR), '; ' ORDER BY CAST(l.{server_col} AS VARCHAR))
              AS most_recent_server_names
        FROM long_full l
        JOIN max_dt m
          ON l.dup_group_id_full = m.dup_group_id_full
         AND l.dt = m.max_dt
        GROUP BY 1
    """)

    # c) parent index
    parent_path = os.path.join(out_dir, parent_index_name)
    con.execute(f"""
        COPY (
            SELECT
                l.dup_group_id_full,

                l.parent_record_id_selected AS parent_record_id,
                l.parent_server_name_selected AS parent_server_name,
                l.parent_doi_selected       AS parent_doi,
                l.parent_url_selected       AS parent_url,
                l.parent_title_selected     AS parent_title,
                l.parent_authors_selected   AS parent_authors,
                l.parent_date_selected      AS parent_date_first_seen,
                l.parent_year_selected      AS parent_year_first_seen,

                STRING_AGG(DISTINCT CAST(l.{record_id_col} AS VARCHAR), '; '
                           ORDER BY CAST(l.{record_id_col} AS VARCHAR)) AS version_record_ids,

                -- cleaned DOI list (sorted, skips null/empty)
                STRING_AGG(l.doi_clean, '; ' ORDER BY l.doi_clean)
                    FILTER (WHERE l.doi_clean IS NOT NULL) AS version_dois,

                s.servers_with_counts,

                MAX(l.group_size) AS total_versions,
                MIN(l.dt) AS first_version_date,
                MAX(l.dt) AS last_version_date,

                mr.most_recent_server_names AS most_recent_version_servers

            FROM long_full l
            LEFT JOIN servers_counts_str s USING(dup_group_id_full)
            LEFT JOIN most_recent_servers mr USING(dup_group_id_full)
            GROUP BY
                l.dup_group_id_full,
                l.parent_record_id_selected,
                l.parent_server_name_selected,
                l.parent_doi_selected,
                l.parent_url_selected,
                l.parent_title_selected,
                l.parent_authors_selected,
                l.parent_date_selected,
                l.parent_year_selected,
                s.servers_with_counts,
                mr.most_recent_server_names
        )
        TO '{parent_path}' (FORMAT 'parquet');
    """)


    # -------------
    # 3) Metrics
    # -------------
    total_rows = con.execute("SELECT COUNT(*) FROM base").fetchone()[0]
    clustered_rows = con.execute(f"SELECT COUNT(*) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
    singleton_rows = total_rows - clustered_rows

    clusters = con.execute(f"SELECT COUNT(DISTINCT CAST({group_id_col} AS VARCHAR)) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
    groups_full = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM base").fetchone()[0]

    # sanity checks
    long_rows = con.execute("SELECT COUNT(*) FROM long_full").fetchone()[0]
    parent_rows = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM long_full").fetchone()[0]

    metrics = {
        "total_rows": int(total_rows),
        "clustered_rows_has_dup_group_id": int(clustered_rows),
        "singleton_rows_no_dup_group_id": int(singleton_rows),
        "clusters_distinct_dup_group_id": int(clusters),
        "groups_full_clusters_plus_singletons": int(groups_full),
        "long_full_rows_written": int(long_rows),
        "parent_index_rows_written": int(parent_rows),
    }

    # write metrics json
    metrics_path = os.path.join(out_dir, metrics_name)
    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)

    return {
        "long_full_path": long_path,
        "parent_index_path": parent_path,
        "metrics_path": metrics_path,
        "metrics": metrics,
    }


In [5]:
result = build_parent_index_and_long_full_duckdb(
    df=data,          # or data_out after unify steps
    out_dir="outputs_new/parent",
    group_id_col="dup_group_id",  # or "dup_group_id_merged" if you renamed it
    date_col="date_first_seen",
)

result["metrics"]


{'total_rows': 8084860,
 'clustered_rows_has_dup_group_id': 1113404,
 'singleton_rows_no_dup_group_id': 6971456,
 'clusters_distinct_dup_group_id': 495664,
 'groups_full_clusters_plus_singletons': 7467120,
 'long_full_rows_written': 8084860,
 'parent_index_rows_written': 7467120}

In [6]:
result

{'long_full_path': 'outputs_new/parent/dedupe_clusters_long_full.parquet',
 'parent_index_path': 'outputs_new/parent/parent_version_index_full.parquet',
 'metrics_path': 'outputs_new/parent/dedupe_full_metrics.json',
 'metrics': {'total_rows': 8084860,
  'clustered_rows_has_dup_group_id': 1113404,
  'singleton_rows_no_dup_group_id': 6971456,
  'clusters_distinct_dup_group_id': 495664,
  'groups_full_clusters_plus_singletons': 7467120,
  'long_full_rows_written': 8084860,
  'parent_index_rows_written': 7467120}}

In [7]:
import duckdb

con = duckdb.connect()

long_path = "outputs_new/parent/dedupe_clusters_long_full.parquet"
parent_path = "outputs_new/parent/parent_version_index_full.parquet"


In [8]:
long_path

'outputs_new/parent/dedupe_clusters_long_full.parquet'

In [9]:
parent_path

'outputs_new/parent/parent_version_index_full.parquet'

# exploration

In [10]:
len(long_path)

52

In [11]:
len(parent_path)

52

In [12]:
juan = con.sql(f"""
SELECT *
FROM '{parent_path}'
WHERE total_versions > 1
ORDER BY RANDOM()
LIMIT 10
""").df()

juan#.shape[0]

,dup_group_id_full,parent_record_id,parent_server_name,parent_doi,parent_url,parent_title,parent_authors,parent_date_first_seen,parent_year_first_seen,version_record_ids,version_dois,servers_with_counts,total_versions,first_version_date,last_version_date,most_recent_version_servers
0,openalex::W3124001418,openalex::W3124001418,RePEc: Research Papers in Economics,None,http://econpapers.repec.org/paper/sancdmawp/05...,The Impact of Simple Fiscal Rules in Growth Mo...,Sugata Ghosh; Charles R. Nolan,2005-01-15,2005,crossref::10.2139/ssrn.717941; openalex::W3124...,10.2139/ssrn.717941,RePEc: Research Papers in Economics (1); SSRN (1),2,2005-01-15,2005-05-09,SSRN
1,datacite::10.5281/zenodo.17596018,datacite::10.5281/zenodo.17596017,Zenodo,10.5281/zenodo.17596017,https://zenodo.org/doi/10.5281/zenodo.17596017,Community-Driven Survival Analysis of Penile I...,principles,2025-11-13,2025,datacite::10.5281/zenodo.17596017; datacite::1...,10.5281/zenodo.17596017; 10.5281/zenodo.175960...,Zenodo (3),3,2025-11-13,2025-11-13,Zenodo
2,datacite::10.13140/rg.2.2.19358.28488,datacite::10.13140/rg.2.2.19358.28488,ResearchGate,10.13140/rg.2.2.19358.28488,https://www.researchgate.net/doi/10.13140/RG.2...,Reconstruction of traffic speed distributions ...,"Herty, Michael; Tosin, Andrea; Visconti, Giuse...",2019-12-04,2019,datacite::10.13140/rg.2.2.19358.28488; datacit...,10.13140/rg.2.2.19358.28488; 10.48550/arxiv.19...,ResearchGate (1); arXiv (1),2,2019-12-04,2022-02-26,arXiv
3,datacite::10.48550/arxiv.2003.12619,datacite::10.48550/arxiv.2003.12619,arXiv,10.48550/arxiv.2003.12619,https://arxiv.org/abs/2003.12619,Sharp well-posedness for a coupled system of m...,"Carvajal, Xavier; Esquivel, Liliana; Santos, R...",2022-02-25,2022,datacite::10.48550/arxiv.1810.03066; datacite:...,10.48550/arxiv.1810.03066; 10.48550/arxiv.2003...,arXiv (2),2,2022-02-25,2022-03-01,arXiv
4,crossref::10.12688/f1000research.14848.1,crossref::10.12688/f1000research.14848.1,F1000Research,10.12688/f1000research.14848.1,https://f1000research.com/articles/7-724/v1,Case Report: Large ileal intramural hematoma p...,"Alizadeh Forutan, Maryam; Mojarad, Fereshteh A...",2018-06-11,2018,crossref::10.12688/f1000research.14848.1; cros...,10.12688/f1000research.14848.1; 10.12688/f1000...,F1000Research (2),2,2018-06-11,2019-12-30,F1000Research
5,crossref::10.2139/ssrn.1822333,crossref::10.2139/ssrn.1822333,SSRN,10.2139/ssrn.1822333,http://www.ssrn.com/abstract=1822333,Robust Capital Regulation,"Acharya, Viral V.; Mehran, Hamid; Schuermann, ...",2011-04-25,2011,crossref::10.2139/ssrn.1822333; crossref::10.2...,10.2139/ssrn.1822333; 10.2139/ssrn.2070102,SSRN (2); EconStor Preprints (1),3,2011-04-25,2012-05-30,SSRN
6,crossref::10.2139/ssrn.3962896,crossref::10.2139/ssrn.3962896,SSRN,10.2139/ssrn.3962896,https://www.ssrn.com/abstract=3962896,Epidemic Analysis of Peste Des Petits Ruminant...,"Zhang, Shuwen; Liang, Ruirui; Qiu, Songyin; Zh...",2021-12-09,2021,crossref::10.21203/rs.3.rs-1288195/v1; crossre...,10.21203/rs.3.rs-1288195/v1; 10.2139/ssrn.3962896,Research Square (1); SSRN (1),2,2021-12-09,2022-02-21,Research Square
7,crossref::10.21203/rs.2.19971/v1,crossref::10.21203/rs.2.19971/v1,Research Square,10.21203/rs.2.19971/v1,https://www.researchsquare.com/article/rs-1058...,Comparison of transversus abdominis plan block...,"Kacan, Merve; Gulec, Handan; kosar, berrin; il...",2020-01-03,2020,crossref::10.21203/rs.2.19971/v1; crossref::10...,10.21203/rs.2.19971/v1; 10.21203/rs.2.19971/v2,Research Square (2),2,2020-01-03,2020-01-28,Research Square
8,crossref::10.32388/0hpphh,crossref::10.32388/0hpphh,Qeios,10.32388/0hpphh,https://www.qeios.com/read/0HPPHH,Remove positive social consequence for outcome...,Behaviour Change Intervention Ontology (BCIO),2023-06-04,2023,crossref::10.32388/0hpphh; crossref::10.32388/...,10.32388/0hpphh; 10.32388/0hpphh.2; 10.32388/0...,Qeios (3),3,2023-06-04,2024-05-13,Qeios
9,datacite::10.17605/osf.io/jmvt5,crossref::10.31235/osf.io/jmvt5,SocArXiv,10.31235/osf.io/j

In [13]:
juan = con.sql(f"""
SELECT *
FROM '{parent_path}'
WHERE total_versions > 1
ORDER BY RANDOM()
LIMIT 10
""").df()

juan#.shape[0]

,dup_group_id_full,parent_record_id,parent_server_name,parent_doi,parent_url,parent_title,parent_authors,parent_date_first_seen,parent_year_first_seen,version_record_ids,version_dois,servers_with_counts,total_versions,first_version_date,last_version_date,most_recent_version_servers
0,crossref::10.2196/preprints.36935,crossref::10.2196/preprints.36935,JMIR Preprints,10.2196/preprints.36935,http://preprints.jmir.org/preprint/36935,Efficiency and Effectiveness of Smartphone App...,"Fadinie, Wulan",2022-01-31,2022,crossref::10.2196/preprints.36935; crossref::1...,10.2196/preprints.36935; 10.2196/preprints.369...,JMIR Preprints (3),3,2022-01-31,2022-01-31,JMIR Preprints
1,datacite::10.5281/zenodo.13893191,datacite::10.5281/zenodo.13893191,Zenodo,10.5281/zenodo.13893191,https://zenodo.org/doi/10.5281/zenodo.13893191,"Fractales del Rombo, del Sello de Salomón y de...","Soltero Escobar, José Fabián",2024-10-05,2024,datacite::10.5281/zenodo.13893191; datacite::1...,10.5281/zenodo.13893191; 10.5281/zenodo.13893192,Zenodo (2),2,2024-10-05,2024-10-05,Zenodo
2,openalex::W3163652297,openalex::W3163652297,HAL,10.34692/7jhh-gj76,https://inrap.hal.science/hal-03215728,Le diagnostic archéologique : un atout pour la...,Florie Spiès; Perrine Toussaint; Geertrui Blan...,2021-04-30,2021,openalex::W3154871505; openalex::W3163652297,10.34692/7jhh-gj76; 10.34692/ry19-2143,HAL (2),2,2021-04-30,2025-09-19,HAL
3,crossref::10.12688/f1000research.9372.1,crossref::10.12688/f1000research.9372.1,F1000Research,10.12688/f1000research.9372.1,https://f1000research.com/articles/5-1964/v1,South-East Asian strains of Plasmodium falcipa...,"Singh, Gajinder Pal; Sharma, Amit",2016-08-12,2016,crossref::10.12688/f1000research.9372.1; cross...,10.12688/f1000research.9372.1; 10.12688/f1000r...,F1000Research (2),2,2016-08-12,2016-10-21,F1000Research
4,openalex::W3122123589,openalex::W3122123589,Munich Personal RePEc Archive,None,None,A new class of production functions and an arg...,Jakub Growiec,2006-06-19,2006,crossref::10.2139/ssrn.926522; openalex::W3122...,10.2139/ssrn.926522,Munich Personal RePEc Archive (1); SSRN (1),2,2006-06-19,2006-08-28,SSRN
5,datacite::10.48550/arxiv.1811.11583,datacite::10.48550/arxiv.1811.11583,arXiv,10.48550/arxiv.1811.11583,https://arxiv.org/abs/1811.11583,NIEL DOSE and DLTS Analyses on Triple and Sing...,"Campesato, Roberta; Baur, Carsten; Casale, Mar...",2022-03-01,2022,datacite::10.48550/arxiv.1704.06495; datacite:...,10.48550/arxiv.1704.06495; 10.48550/arxiv.1811...,arXiv (2),2,2022-03-01,2022-03-04,arXiv
6,crossref::10.21203/rs.3.rs-64829/v1,crossref::10.21203/rs.3.rs-64829/v1,Research Square,10.21203/rs.3.rs-64829/v1,https://www.researchsquare.com/article/rs-6482...,Analysis of cesarean section rates using Robso...,"Abubeker, Ferid A.; Gashawbeza, Biruck; Gebre,...",2020-09-01,2020,crossref::10.21203/rs.3.rs-64829/v1; crossref:...,10.21203/rs.3.rs-64829/v1; 10.21203/rs.3.rs-64...,Research Square (3),3,2020-09-01,2020-12-16,Research Square
7,datacite::10.48550/arxiv.2006.08079,datacite::10.48550/arxiv.2006.08079,arXiv,10.48550/arxiv.2006.08079,https://arxiv.org/abs/2006.08079,Regularized finite difference methods for the ...,"Yan, Jingye; Zhang, Hong; Qian, Xu; Song, Songhe",2022-02-24,2022,datacite::10.48550/arxiv.2006.08079; datacite:...,10.48550/arxiv.2006.08079; 10.48550/arxiv.2006...,arXiv (2),2,2022-02-24,2022-02-24,arXiv
8,crossref::10.1002/essoar.10511492.1,crossref::10.1002/essoar.10511492.1,Earth and Space Science Open Archive,10.1002/essoar.10511492.1,https://essopenarchive.org/doi/full/10.1002/es...,Temperature dependence of clumped isotopes (∆4...,"de Winter, Niels Jonathan; Witbaard, Rob; Kock...",2022-05-31,2022,crossref::10.1002/essoar.10511492.1; crossref:...,10.1002/essoar.10511492.1; 10.1002/essoar.1051...,Earth and Space Science Open Archive (2),2,2022-05-31,2022-08-25,Earth and Space Science Open Archive
9,openalex::W3124652205,openalex::W3124652205,RePEc: Research Papers in Economics,None,https://econpapers.repec.

## to look at

- same preprint posted to two servers same day (like RS, posted automatically)
- same preprint at datacite and crossref (moved over from one agency to the other?)

In [14]:
juan.iloc[0].version_record_ids

'crossref::10.2196/preprints.36935; crossref::10.2196/preprints.36939; crossref::10.2196/preprints.36940'

In [15]:
con.sql(f"""
SELECT *
FROM '{long_path}'
ORDER BY RANDOM()
LIMIT 10
""").df()


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup,dup_group_id_full,dt,rid_num,doi_clean,server_clean,parent_record_id_selected,parent_server_name_selected,parent_doi_selected,parent_url_selected,parent_title_selected,parent_authors_selected,parent_date_selected,parent_year_selected,is_parent_selected,group_size
0,openalex::W4393583828,HAL,openalex,None,None,https://hal.laas.fr/tel-01930602,Development of a wireless pressure measurement...,Jérémie Fourmann,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2017-12-05,2017,parent,None,None,fourmann_jeremie,development of a wireless pressure measurement...,fourmann|j,None,parent,None,None,parent,None,None,openalex::W4393583828,2017-12-05,<NA>,None,HAL,openalex::W4393583828,HAL,None,https://hal.laas.fr/tel-01930602,Development of a wireless pressure measurement...,Jérémie Fourmann,2017-12-05,2017,True,1
1,openalex::W1801852471,RePEc: Research Papers in Economics,openalex,None,None,https://EconPapers.repec.org/RePEc:dar:wpaper:...,Bestimmt der Kunde das Produkt,Günter Specht,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2008-11-19,2008,parent,None,None,gunter_specht,bestimmt der kunde das produkt,specht|g,None,parent,None,None,parent,None,None,openalex::W1801852471,2008-11-19,<NA>,None,RePEc: Research Papers in Economics,openalex::W1801852471,RePEc: Research Papers in Economics,None,https://EconPapers.repec.org/RePEc:dar:wpaper:...,Bestimmt der Kunde das Produkt,Günter Specht,2008-11-19,2008,True,1
2,datacite::10.48550/arxiv.0901.0950,arXiv,datacite,10.48550/arxiv.0901.0950,https://doi.org/10.48550/arxiv.0901.0950,https://arxiv.org/abs/0901.0950,Ab initio many-body calculations of nucleon-nu...,"Quaglioni, Sofia; Navratil, Petr","LLNL, Livermore",None,"[{""relatedIdentifier"": ""10.1103/physrevc.79.04...",1,,10.1103/physrevc.79.044606,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-15,2022,parent,None,None,navratil_petr;quaglioni_sofia,ab initio many body calculations of nucleon nu...,navratil|p;quaglioni|s,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.0901.0950,2022-03-15,<NA>,10.48550/arxiv.0901.0950,arXiv,datacite::10.48550/arxiv.0901.0950,arXiv,10.48550/arxiv.0901.0950,https://arxiv.org/abs/0901.0950,Ab initio many-body calculations of nucleon-nu...,"Quaglioni, Sofia; Navratil, Petr",2022-03-15,2022,True,1
3,openalex::W4296301094,HAL,openalex,None,None,https://hal.science/hal-00395843,Growth and Characterization of Silicon Nanowir...,F. Dhalluin; T. Baron; P. Gentile; Karim Aisso...,Laboratoire des Technologies de la Microélectr...,FR,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2005-01-01,2005,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,openalex::W4296301094,2005-01-01,<NA>,None,HAL,openalex::W4296301094,HAL,None,https://hal.science/hal-00395843,Growth and Characterization of Silicon Nanowir...,F. Dhalluin; T. Baron; P. Gentile; Karim Aisso...,2005-01-01,2005,True,1
4,openalex::W3122414835,RePEc: Research Papers in Economics,openalex,None,None,https://econpapers.repec.org/RePEc:pra:mprapa:...,"Comparative Analysis of Theory X, Theory Y, Th...",P. S. Aithal; Suresh Kumar,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2016-01-01,2016,parent,None,None,aithal_p_s;kuma

In [16]:
juan = con.sql(f"""
SELECT dup_group_id_full, server_name, COUNT(*) AS n
FROM '{long_path}'
GROUP BY 1, 2
ORDER BY n DESC
LIMIT 1000
""").df()

juan.groupby('server_name').size().nlargest(100)

server_name
Zenodo                                  534
AgEcon Search                           138
Preprints.org                            56
ResearchGate                             50
Open Science Framework                   35
SSRN                                     22
Cambridge Open Engage                    19
Qeios                                    17
PsyArXiv                                 16
INA-Rxiv                                 13
ChemRxiv                                 13
Research Square                          12
Humanities Commons CORE                   9
Thesis Commons                            8
HAL                                       8
Authorea Inc.                             7
TechRxiv                                  7
Covid-19 Preprints                        6
arXiv                                     5
RePEc: Research Papers in Economics       4
Nature Precedings                         3
UCL Open Environment                      2
Earth and Space Scie

In [17]:
con.sql(f"""
SELECT *
FROM '{long_path}'
WHERE dup_group_id_full IN (
    SELECT DISTINCT dup_group_id_full
    FROM '{long_path}'
    ORDER BY RANDOM()
    LIMIT 5
)
""").df()


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup,dup_group_id_full,dt,rid_num,doi_clean,server_clean,parent_record_id_selected,parent_server_name_selected,parent_doi_selected,parent_url_selected,parent_title_selected,parent_authors_selected,parent_date_selected,parent_year_selected,is_parent_selected,group_size
0,openalex::W3125224365,RePEc: Research Papers in Economics,openalex,None,None,https://ideas.repec.org/p/ris/kiepwp/2014_005....,Regional Financial Arrangement in East Asia: P...,Pravin Krishna; Jiyoung Choi; Tae Hoon Lim,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2014-01-01,2014,parent,None,openalex::W3125224365,choi_jiyoung;hoon_lim_tae;krishna_pravin,regional financial arrangement in east asia po...,None,None,parent,None,13876040202262996357,parent,None,13876040202262996357,openalex::W3125224365,2014-01-01,<NA>,None,RePEc: Research Papers in Economics,openalex::W3125224365,RePEc: Research Papers in Economics,None,https://ideas.repec.org/p/ris/kiepwp/2014_005....,Regional Financial Arrangement in East Asia: P...,Pravin Krishna; Jiyoung Choi; Tae Hoon Lim,2014-01-01,2014,True,2
1,crossref::10.2139/ssrn.2634318,SSRN,crossref,10.2139/ssrn.2634318,https://doi.org/10.2139/ssrn.2634318,http://www.ssrn.com/abstract=2634318,Regional Financial Arrangement in East Asia: P...,"Krishna, Pravin; Choi, Jiyoung; Lim, Tae Hoon",None,None,None,None,,,,,false,<NA>,<NA>,None,None,parent - duplicate (SSRN),2015-07-23,2015,parent,openalex::W3125224365,openalex::W3125224365,choi_jiyoung;hoon_lim_tae;krishna_pravin,regional financial arrangement in east asia po...,None,None,parent - duplicate (RePEc: Research Papers in ...,openalex::W3125224365,13876040202262996357,parent - duplicate (RePEc: Research Papers in ...,openalex::W3125224365,13876040202262996357,openalex::W3125224365,2015-07-23,<NA>,10.2139/ssrn.2634318,SSRN,openalex::W3125224365,RePEc: Research Papers in Economics,None,https://ideas.repec.org/p/ris/kiepwp/2014_005....,Regional Financial Arrangement in East Asia: P...,Pravin Krishna; Jiyoung Choi; Tae Hoon Lim,2014-01-01,2014,False,2
2,datacite::10.48550/arxiv.alg-geom/9607021,arXiv,datacite,10.48550/arxiv.alg-geom/9607021,https://doi.org/10.48550/arxiv.alg-geom/9607021,https://arxiv.org/abs/alg-geom/9607021,Embeddings of curves and surfaces,"Catanese, F.; Franciosi, M.; Hulek, K.; Reid, M.",None,None,[],1,,,,,false,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-19,2022,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.alg-geom/9607021,2022-03-19,<NA>,10.48550/arxiv.alg-geom/9607021,arXiv,datacite::10.48550/arxiv.alg-geom/9607021,arXiv,10.48550/arxiv.alg-geom/9607021,https://arxiv.org/abs/alg-geom/9607021,Embeddings of curves and surfaces,"Catanese, F.; Franciosi, M.; Hulek, K.; Reid, M.",2022-03-19,2022,True,1
3,datacite::10.48550/arxiv.1709.10252,arXiv,datacite,10.48550/arxiv.1709.10252,https://doi.org/10.48550/arxiv.1709.10252,https://arxiv.org/abs/1709.10252,Neutrino Parameters from Reactor and Accelerat...,"Lindner, Manfred; Rodejohann, Werner; Xu, Xun-Jie",None,None,"[{""relatedIdentifier"": ""10.1103/physrevd.97.07...",2,,10.1103/physrevd.97.075024,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-04,2022,parent,None,None,jie_